# Investitage the current status

Store a game save from which you want to take constraints for the optimal production.

In this example, the factory is at a state after completing Phase 2. It looks like this (visualization via [Satisfactor Calculator](https://satisfactory-calculator.com/en/interactive-map):

![Production Map](./docs/Assistory-phase-2-completed.png))

In [1]:
STATS_FOLDER = 'example/example_data'
# Extract the stats (TODO: set the save file path and uncomment the script)
# START_SAV_FILE = '<your_save_file>.sav'
# !python3 main_game_stats.py {START_SAV_FILE} --print-progress --out {START_STATS_FOLDER}

We can investigate some stats:

In [2]:
from assistory import game

base_item_rate = game.ItemValues.load(STATS_FOLDER + '/base_item_rate.yml')
print('Existing item balance:')
base_item_rate.pprint(0)

Existing item balance:
Desc_Cable_C               30.0
Desc_Cement_C              20.0
Desc_IronPlateReinforced_C 10.5
Desc_Rotor_C               10.0
Desc_SpaceElevatorPart_2_C 10.0
Desc_Stator_C              1.0
Desc_SteelPipe_C           8.25
Desc_Wire_C                22.0


In [3]:
base_recipe_count = game.RecipeValues.load(STATS_FOLDER + '/base_recipe_count.yml')
print('Existing recipes:')
base_recipe_count.round(3).pprint(0)

Existing recipes:
Recipe_Alternate_IngotSteel_1_C          4.5
Recipe_Alternate_ReinforcedIronPlate_2_C 3.2
Recipe_Alternate_Screw_C                 4.8
Recipe_Alternate_SteelRod_C              1.094
Recipe_Cable_C                           1.0
Recipe_Concrete_C                        1.333
Recipe_GeneratorCoalCoal_C               12.0
Recipe_IngotCopper_C                     3.5
Recipe_IngotIron_C                       12.0
Recipe_IronPlate_C                       3.0
Recipe_IronRod_C                         2.0
Recipe_MinerMk1Coal_C                    6.0
Recipe_MinerMk1OreCopper_C               1.75
Recipe_MinerMk1OreIron_C                 6.0
Recipe_MinerMk1Stone_C                   1.0
Recipe_ModularFrame_C                    2.5
Recipe_Rotor_C                           2.5
Recipe_Screw_C                           0.25
Recipe_SpaceElevatorPart_2_C             2.0
Recipe_Stator_C                          0.2
Recipe_SteelBeam_C                       4.0
Recipe_SteelPipe_C           

In [4]:
occupied_resource_nodes = game.ResourceNodeValues.load(STATS_FOLDER + '/occupied_resource_nodes.yml')
print('Occupied resource nodes:')
occupied_resource_nodes.round(2).pprint(0)

Occupied resource nodes:
Desc_Coal_C-non_fracking      6.0
Desc_OreCopper_C-non_fracking 1.75
Desc_OreIron_C-non_fracking   6.0
Desc_Stone_C-non_fracking     1.0


# Generate the plan
First, the objective and constraints are set. In this example, a production of motors should be build after completing phase 2.

In [5]:
static_production_config = dict()
static_production_config['maximize_sell_rate'] = True
static_production_config['weights_sell_rate'] = {
    'Desc_Motor_C': 1 # only sell rate of motor is maximized now
}


It can be observed that one of the copper miners is underclocked:
```
Recipe_MinerMk1OreCopper_C               1.75
```
It is a pure node and the miner runs at 87.5%, so, 0.25 copper nodes units are still free.

All Miners in the current state are only Mk1. Using Mk2 the occupied nodes can provide 100% additional capacity.

In [6]:
available_resouce_nodes = occupied_resource_nodes.copy()
available_resouce_nodes['Desc_OreCopper_C-non_fracking'] = 2.0
static_production_config['available_resource_nodes'] = available_resouce_nodes.as_dict_ignoring(0)

Still 180MW power capacity is unused

In [7]:
static_production_config['base_power'] = 180

Next, the configuration is written to file and the optimization is executed

In [8]:
import yaml

STATIC_PROD_CONF ='example/example_configurations/static_production_config.yml'
with open(STATIC_PROD_CONF, 'w') as fp:
    yaml.safe_dump(static_production_config, fp, indent=2)

user_config = {
    'base_item_rate_file': 'example/example_data/base_item_rate.yml',
    'static_production_config_file': STATIC_PROD_CONF,
    'unlocked_recipes_file': 'example/example_data/unlocked_recipes.yml',
}
OPTIM_PROD_USER_CONF = 'example/example_configurations/optimal_production_user_config.yml'
with open(OPTIM_PROD_USER_CONF, 'w') as fp:
    yaml.safe_dump(user_config, fp, indent=2)

OPTIM_PLAN_OUT_FILE = 'data/phase-3-motors.yml'
!python3 main_optimal_production.py {OPTIM_PROD_USER_CONF} --out {OPTIM_PLAN_OUT_FILE}


Power balance (MW):
FoundryMk1(9.863) = -157.808
SmelterMk1(32.0) = -128.0
ConstructorMk1(104.159) = -416.636
GeneratorCoal(21.699) = 1627.425
MinerMk2(14.0) = -210.0
AssemblerMk1(48.815) = -732.225
WaterPump(8.137) = -162.74
Total balance: -179.984 MW

Required resource nodes:
Desc_Coal_C-non_fracking      6.0
Desc_OreCopper_C-non_fracking 2.0
Desc_OreIron_C-non_fracking   6.0

Sold items (/min):
Cable = 30.0
Cement = 20.0
IronPlateReinforced = 10.5
Motor = 46.832
SpaceElevatorPart_2 = 10.0

Objective value = 46.832

Problem solved in 33 milliseconds


That is a huge plan. Maybe let's constrain the production to only use the existing power.

In [9]:
recipes_unlocked = game.RecipeFlags.load('example/example_data/unlocked_recipes.yml')
recipes_unlocked -= set(
    recipe_name
    for recipe_name in recipes_unlocked
    if 'Generator' in recipe_name
)
static_production_config['unlocked_recipes'] = sorted(recipes_unlocked)

with open(STATIC_PROD_CONF, 'w') as fp:
    yaml.safe_dump(static_production_config, fp, indent=2)

# unlocked recipes are now defined via STATIC_PROD_CONF file
user_config = {
    'base_item_rate_file': 'example/example_data/base_item_rate.yml',
    'static_production_config_file': STATIC_PROD_CONF,
}
with open(OPTIM_PROD_USER_CONF, 'w') as fp:
    yaml.safe_dump(user_config, fp, indent=2)

In [10]:
!python3 main_optimal_production.py {OPTIM_PROD_USER_CONF} --out {OPTIM_PLAN_OUT_FILE}


Power balance (MW):
MinerMk1(2.931) = -14.655
ConstructorMk1(9.062) = -36.248
FoundryMk1(1.075) = -17.2
SmelterMk1(4.428) = -17.712
AssemblerMk1(6.279) = -94.185
Total balance: -180.0 MW

Required resource nodes:
Desc_Coal_C-non_fracking      0.717
Desc_OreCopper_C-non_fracking 0.838
Desc_OreIron_C-non_fracking   1.376

Sold items (/min):
Cable = 30.0
Cement = 20.0
IronPlateReinforced = 10.5
Motor = 8.163
SpaceElevatorPart_2 = 10.0

Objective value = 8.163

Problem solved in 24 milliseconds


# Execute the plan
The optimized recipes can not be implemented by building the suitable factories.

Therefore the building costs need to be collected

In [11]:
import math

recipe_plan = game.RecipeValues.load(OPTIM_PLAN_OUT_FILE)

full_recipes = game.RecipeValues(
    {   
        recipe_name: math.ceil(amount)
        for recipe_name, amount in recipe_plan.round(10).items()
    }
)
buildings_plan = full_recipes.get_buildings()
investment_costs = buildings_plan.get_costs()

print('\nBuildings to build:')
buildings_plan.round(3).pprint(0)
print('\nInvestment costs:')
investment_costs.round(3).pprint(0)


Buildings to build:
Desc_AssemblerMk1_C   8
Desc_ConstructorMk1_C 11
Desc_FoundryMk1_C     2
Desc_MinerMk1_C       4
Desc_SmelterMk1_C     5

Investment costs:
BP_ItemDescriptorPortableMiner_C 4
Desc_Cable_C                     168
Desc_Cement_C                    80
Desc_IronPlateReinforced_C       86
Desc_IronPlate_C                 40
Desc_IronRod_C                   25
Desc_ModularFrame_C              20
Desc_Rotor_C                     52
Desc_Wire_C                      40


Use the graph export to generate a visual representation of the plan. This helps to plan
- where which factory should be placed
- how to to connect factories

In [12]:
!python3 main_export_graph.py {OPTIM_PLAN_OUT_FILE} --out data/phase-3-motors.gexf

Opening the file **data/graphs/phase-3-motors.gexf** and adjust the appearance:
- optimize the layout, e.g. with the Yifan Hu algorithm
- color the nodes by partition encoded in the attribute `node_type`
- Add node and edge description from label

The graph visualized with [Gephi](https://gephi.org/) looks as follows

![Visualized Recipe Plan](./docs/optimal_production_graph.png)